<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-31 15:14:36
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.43 C
-------------------
Today PnL: 1.19 L (0.84%)
Current PnL: -26.02 L (-16.59%)
CY Booked + Current PnL: -11.73 L (-7.48%)
-------------------
Total profit:  1.52 L
Total loss:  -27.54 L
-------------------
Total Booked + Current PnL: 15.25 L (11.75%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.99%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.13 L (63.71%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.86,-19.70,24.66,0.09,19955.0,-19858.0,80919.0,87.83,39.0,M-SC,3.18,253.0,-1.00,0.57,6.31,OX40N,NTT,DURABLES
52,NESTLEIND,1377.00,1.09,13.55,6.91,21.40,20826.0,35966.0,301392.0,5.31,66.0,X-LC,5.20,12.0,1.73,2.11,20.73,XY25,NTT,FMCG
50,MASFIN,398.61,2.72,-1.10,23.41,22.05,22684.0,-1080.0,96900.0,-14.63,56.0,H-SC,6.60,164.0,-0.05,0.68,40.85,XR,ATH,FINANCE
83,VOLTAS,1530.00,0.80,6.51,12.38,19.69,25283.0,12483.0,204225.0,-3.84,47.0,X-MC,2.13,78.0,0.49,1.43,13.75,XY25,NTT,AC
44,ITC,452.00,0.64,-1.19,12.12,10.78,28780.0,-2863.0,237455.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,3.98,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,GLAXO,3466.20,3.71,-0.74,40.18,39.15,79479.0,-1476.0,197808.0,-20.38,43.0,X-MC,10.73,60.0,-0.02,1.38,28.05,X40,ATH,PHARMA
65,SHALBY,327.00,3.65,-22.07,60.52,25.10,85437.0,-39972.0,141171.0,938.28,50.0,M-SC,9.12,250.0,-0.47,0.99,13.13,XY24,NTT,HEALTHCARE
10,BAJAJHLDNG,14451.52,3.26,1.40,27.44,29.22,52899.0,2655.0,192780.0,-1.33,51.0,X-LC,12.16,35.0,0.05,1.35,10.71,X40,ATH,FINANCE
51,MEDANTA,1486.00,2.92,-1.06,25.41,24.08,31915.0,-1351.0,125599.0,-9.02,50.0,X-SC,7.60,89.0,-0.04,0.88,18.34,XY24,NTT,HEALTHCARE
50,MASFIN,398.61,2.72,-1.10,23.41,22.05,22684.0,-1080.0,96900.0,-14.63,56.0,H-SC,6.60,164.0,-0.05,0.68,40.85,XR,ATH,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-4.24,-64.47,181.67,0.07,88535.0,-88443.0,48734.0,1686.62,42.0,L-SC,4.35,268.0,-1.00,0.34,21.20,OX40N,NTT,JEWELLERY
19,COFFEEDAY,80.00,-2.91,-38.48,119.90,35.27,83750.0,-43699.0,69850.0,-53.72,53.0,L-SC,7.05,270.0,-0.52,0.49,70.96,XR,NTT,HOTELS
55,QUESS,424.00,-2.14,-30.67,106.03,42.84,47788.0,-19936.0,45070.0,-54.82,40.0,X-SC,31.63,83.0,-0.42,0.32,1.97,XY24,NTT,MISC
8,AWL,485.00,-1.33,-25.06,104.81,53.50,237022.0,-75607.0,226144.0,-63.76,33.0,X-MC,6.98,58.0,-0.32,1.58,0.45,XY24,NTT,FMCG
74,TCS,4389.96,-1.32,-11.91,37.02,20.71,123353.0,-45031.0,333206.0,-24.27,47.0,X-LC,3.01,1.0,-0.37,2.33,11.34,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.63,-0.45,82.5,81.67,77044.0,-423.0,93387.0,8411.11,66.0,L-SC,16.04,271.0,-0.01,0.65,80.85,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,2.28,-5.20,119.31,107.91,166998.0,-7674.0,139970.0,-22.98,45.0,M-SC,11.59,216.0,-0.05,0.98,4.36,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,1.87,-18.77,23.13,0.02,32797.0,-32765.0,141794.0,115.96,39.0,M-SC,8.36,234.0,-1.00,0.99,23.34,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.86,-19.70,24.66,0.09,19955.0,-19858.0,80919.0,87.83,39.0,M-SC,3.18,253.0,-1.00,0.57,6.31,OX40N,NTT,DURABLES
48,KANSAINER,340.00,0.74,-21.37,44.32,13.47,93972.0,-57636.0,212031.0,-67.73,55.0,H-SC,6.50,158.0,-0.61,1.48,7.81,XY24,NTT,PAINTS
67,SIS,528.00,-0.18,-23.52,58.56,21.26,49726.0,-26117.0,84915.0,2004.15,52.0,H-SC,4.49,166.0,-0.53,0.59,14.73,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.33,7.78,59.83,72.27,109269.0,13184.0,182632.0,-7.44,55.0,M-LC,5.22,99.0,0.12,1.28,14.44,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.33,7.78,59.83,72.27,109269.0,13184.0,182632.0,-7.44,55.0,M-LC,5.22,99.0,0.12,1.28,14.44,XR,NTT,IT
6,ASIANTILES,137.00,0.63,-0.45,82.50,81.67,77044.0,-423.0,93387.0,8411.11,66.0,L-SC,16.04,271.0,-0.01,0.65,80.85,XR,NTT,CERAMICS
39,INDIAMART,4810.62,-1.09,-4.50,116.47,106.72,137181.0,-5554.0,117782.0,-52.73,42.0,H-SC,5.85,139.0,-0.04,0.82,17.82,AR,ATH,MISC
50,MASFIN,398.61,2.72,-1.10,23.41,22.05,22684.0,-1080.0,96900.0,-14.63,56.0,H-SC,6.60,164.0,-0.05,0.68,40.85,XR,ATH,FINANCE
26,FINCABLES,1641.55,2.28,-5.20,119.31,107.91,166998.0,-7674.0,139970.0,-22.98,45.0,M-SC,11.59,216.0,-0.05,0.98,4.36,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,18931.72,-0.76,-21.64,60.89,26.07,93144.0,-42250.0,152971.0,-61.93,24.0,X-MC,15.49,56.0,-0.45,1.07,0.00,X40N,ATH,IT
35,HONAUT,58357.33,-0.50,-19.37,77.76,43.32,102114.0,-31552.0,131320.0,-30.77,27.0,X-SC,7.24,90.0,-0.31,0.92,0.83,X40N,ATH,ELECTRICAL
9,BAJAJHFL,181.50,0.24,-19.14,92.27,55.47,181523.0,-46556.0,196730.0,-28.25,28.0,X-MC,16.45,64.0,-0.26,1.38,0.80,X40N,ATH,FINANCE
8,AWL,485.00,-1.33,-25.06,104.81,53.50,237022.0,-75607.0,226144.0,-63.76,33.0,X-MC,6.98,58.0,-0.32,1.58,0.45,XY24,NTT,FMCG
12,BATAINDIA,2096.00,0.53,-39.17,122.27,35.20,95700.0,-50401.0,78269.0,2.67,35.0,X-SC,17.36,92.0,-0.53,0.55,0.53,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.5,1.10,-17.71,52.52,25.51,80429.0,-32955.0,153140.0,-6.04,40.0,H-LC,0.98,49.0,-0.41,1.07,12.85,AR,ATH,ELECTRICAL
79,UNITDSPR,1644.0,1.40,6.05,13.87,20.77,34441.0,14172.0,248316.0,-3.23,55.0,X-MC,1.59,62.0,0.41,1.74,13.39,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,2.03,-4.48,26.82,21.14,58243.0,-10178.0,217161.0,-11.18,44.0,X-MC,1.62,74.0,-0.17,1.52,20.60,XY24,NTT,PAINTS
21,DABUR,735.0,1.57,-1.57,45.99,43.70,112295.0,-3895.0,244173.0,-9.61,55.0,X-MC,1.98,72.0,-0.03,1.71,13.24,XY24,BTT,FMCG
44,ITC,452.0,0.64,-1.19,12.12,10.78,28780.0,-2863.0,237455.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,3.98,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,18931.72,-0.76,-21.64,60.89,26.07,93144.0,-42250.0,152971.0,-61.93,24.0,X-MC,15.49,56.0,-0.45,1.07,0.00,X40N,ATH,IT
8,AWL,485.00,-1.33,-25.06,104.81,53.50,237022.0,-75607.0,226144.0,-63.76,33.0,X-MC,6.98,58.0,-0.32,1.58,0.45,XY24,NTT,FMCG
12,BATAINDIA,2096.00,0.53,-39.17,122.27,35.20,95700.0,-50401.0,78269.0,2.67,35.0,X-SC,17.36,92.0,-0.53,0.55,0.53,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.36,-26.95,124.70,64.15,216766.0,-64121.0,173830.0,-57.19,36.0,X-SC,5.65,82.0,-0.30,1.22,0.76,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,0.24,-19.14,92.27,55.47,181523.0,-46556.0,196730.0,-28.25,28.0,X-MC,16.45,64.0,-0.26,1.38,0.80,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-2.14,-30.67,106.03,42.84,47788.0,-19936.0,45070.0,-54.82,40.0,X-SC,31.63,83.0,-0.42,0.32,1.97,XY24,NTT,MISC
59,RELAXO,1176.00,0.96,-47.64,190.91,52.31,145090.0,-69161.0,75999.0,-44.25,45.0,X-SC,7.38,91.0,-0.48,0.53,1.73,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.53,-39.17,122.27,35.20,95700.0,-50401.0,78269.0,2.67,35.0,X-SC,17.36,92.0,-0.53,0.55,0.53,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,2.92,-1.06,25.41,24.08,31915.0,-1351.0,125599.0,-9.02,50.0,X-SC,7.60,89.0,-0.04,0.88,18.34,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,-0.50,-19.37,77.76,43.32,102114.0,-31552.0,131320.0,-30.77,27.0,X-SC,7.24,90.0,-0.31,0.92,0.83,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,-1.32,-11.91,37.02,20.71,123353.0,-45031.0,333206.0,-24.27,47.0,X-LC,3.01,1.0,-0.37,2.33,11.34,X40,ATH,IT
42,INFY,1972.00,-0.49,9.85,22.21,34.24,76335.0,30809.0,343697.0,-14.57,52.0,X-LC,7.31,2.0,0.40,2.40,18.94,X40,NTT,IT
76,TMPV,600.00,1.42,-24.80,63.53,22.98,107922.0,-56023.0,169875.0,-21.86,58.0,X-LC,5.83,3.0,-0.52,1.19,6.84,XY24,NTT,AUTO
82,VBL,671.64,0.98,-1.18,37.29,35.67,116386.0,-3732.0,312110.0,-12.96,62.0,X-LC,2.91,4.0,-0.03,2.18,12.51,X40N,ATH,FMCG
44,ITC,452.00,0.64,-1.19,12.12,10.78,28780.0,-2863.0,237455.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,2.63,-36.24,108.42,32.89,53373.0,-27978.0,49228.0,-708.10,60.0,L-MC,6.68,259.0,-0.52,0.34,35.57,XR,NTT,BANKS
6,ASIANTILES,137.00,0.63,-0.45,82.50,81.67,77044.0,-423.0,93387.0,8411.11,66.0,L-SC,16.04,271.0,-0.01,0.65,80.85,XR,NTT,CERAMICS
50,MASFIN,398.61,2.72,-1.10,23.41,22.05,22684.0,-1080.0,96900.0,-14.63,56.0,H-SC,6.60,164.0,-0.05,0.68,40.85,XR,ATH,FINANCE
14,BSOFT,831.70,2.41,-21.35,91.72,50.79,101460.0,-30034.0,110619.0,0.89,55.0,H-SC,11.16,151.0,-0.30,0.77,28.80,XR,ATH,IT
70,SURYODAY,216.00,1.52,-19.54,52.91,23.03,76235.0,-34986.0,144085.0,57.89,54.0,H-SC,6.71,167.0,-0.46,1.01,42.96,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.63,-0.45,82.50,81.67,77044.0,-423.0,93387.0,8411.11,66.0,L-SC,16.04,271.0,-0.01,0.65,80.85,XR,NTT,CERAMICS
14,BSOFT,831.70,2.41,-21.35,91.72,50.79,101460.0,-30034.0,110619.0,0.89,55.0,H-SC,11.16,151.0,-0.30,0.77,28.80,XR,ATH,IT
75,TITAGARH,1548.00,1.43,-16.11,73.72,45.74,149778.0,-39006.0,203171.0,11.66,64.0,H-SC,4.87,174.0,-0.26,1.42,32.14,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.00,2.15,10.69,18.10,30.73,39469.0,21056.0,218061.0,-15.19,70.0,X-MC,4.31,75.0,0.53,1.52,24.82,X40,ATH,INSURANCE
62,ROUTE,2227.21,-0.17,-46.80,217.74,69.05,154151.0,-62270.0,70796.0,-56.81,51.0,H-SC,19.40,140.0,-0.40,0.49,9.18,SR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,44.11
2,50,75.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.34
MC    30.17
LC    26.47
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.47
X40      23.90
X40N     12.62
XY25     10.42
XR        9.62
AR        8.62
OX40N     7.36
SR        0.97
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.75
X-LC    23.09
H-SC    22.68
M-SC    11.29
X-SC     7.89
H-MC     4.71
L-SC     1.48
M-MC     1.37
M-LC     1.28
H-LC     1.07
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.11,-6.66,40.96
IT,12.46,-17.93,77.23
FINANCE,10.86,-14.85,63.05
MISC,6.99,-29.74,82.81
ELECTRICAL,5.82,-12.86,54.20
PAINTS,5.69,-13.32,30.16
INSURANCE,4.70,-0.69,35.15
PHARMA,4.01,-2.39,34.62
AUTO,2.82,-30.06,75.15


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3134331.0,21
AR,1338710.0,10
XR,1284564.0,13
X40,1049531.0,15
X40N,881333.0,9
OX40N,740994.0,10
XY25,402448.0,7
SR,280853.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3546337.0,24
M-SC,1444381.0,15
X-MC,1430089.0,16
X-LC,919975.0,13
X-SC,793303.0,8
H-MC,405317.0,3
L-SC,249329.0,3
M-LC,109269.0,1
H-LC,80429.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237410.0      6
           AR         916222.0      5
M-SC       XY24       815264.0      6
H-SC       XR         782627.0      7
X-MC       X40        479635.0      7
           XY24       407560.0      3
X-LC       X40        389248.0      6
X-MC       X40N       352377.0      4
H-SC       OX40N      329225.0      4
M-SC       OX40N      323234.0      5
X-SC       X40N       316186.0      3
           XY24       296469.0      3
H-SC       SR         280853.0      2
H-MC       AR         214677.0      2
X-LC       X40N       212770.0      2
H-MC       XY24       190640.0      1
X-MC       XY25       190517.0      2
X-LC       XY24       186988.0      2
X-SC       X40        180648.0      2
M-SC       XR         178501.0      2
L-SC       XR         160794.0      2
X-LC       XY25       130969.0      3
M-SC       AR         127382.0      2
M-LC       XR         109269.0      1
L-SC       OX40N       88535.0      1
H-LC       AR          80429.0      1
L-MC       XR          53373.0      1
M-MC       XY25        49170.0      1
L-LC       XY25        31792.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
